In [420]:
import pyspark
from pyspark.sql.functions import *
import time

In [421]:
#gcp_storage_path = "gs://hwk1/White_House_Visitor_Records_Requests.csv"
gcp_storage_path = "gs://hwk1/new_vists.csv"

In [422]:
def load_data(gcp_storage_path, percentage=1.0):
    df = spark.read.csv(gcp_storage_path, header=True)
    
    # Sample a percentage of the data
    if percentage < 1.0:
        df = df.sample(withReplacement=False, fraction=percentage, seed=42)
    
    return df

white_house_visitors = load_data(gcp_storage_path, percentage=1.0)
#white_house_visitors = load_data(gcp_storage_path, percentage=0.5)
#white_house_visitors = load_data(gcp_storage_path, percentage=0.2)
#white_house_visitors = load_data(gcp_storage_path, percentage=0.1)

In [423]:
white_house_visitors.printSchema()

root
 |-- Last Name: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- BDGNBR: string (nullable = true)
 |-- Access Type: string (nullable = true)
 |-- TOA: string (nullable = true)
 |-- POA: string (nullable = true)
 |-- TOD: string (nullable = true)
 |-- POD: string (nullable = true)
 |-- Appointment Made Date: string (nullable = true)
 |-- Appointment Start Date: string (nullable = true)
 |-- Appointment End Date: string (nullable = true)
 |-- Appointment Cancel Date: string (nullable = true)
 |-- Total People: string (nullable = true)
 |-- Last Updated By: string (nullable = true)
 |-- POST: string (nullable = true)
 |-- Last Entry Date: string (nullable = true)
 |-- Terminal Suffix: string (nullable = true)
 |-- Visitee Last Name: string (nullable = true)
 |-- Visitee First Name: string (nullable = true)
 |-- Meeting Location: string (nullable = true)
 |-- Meeting Room: string (n

In [424]:
white_house_visitors.count()

667616

In [425]:
# Have to remove the visitors who never came since cancel
cancelled = white_house_visitors.filter(white_house_visitors['Appointment Cancel Date'].isNotNull())
df = white_house_visitors.filter(white_house_visitors['Appointment Cancel Date'].isNull())

In [426]:
# Replace null values with empty string for 'Last Name', 'First Name', and 'Middle Initial' (case-insensitive)
df = df.withColumn('Last Name', coalesce(col('Last Name'), lit('')))
df = df.withColumn('First Name', coalesce(col('First Name'), lit('')))
df = df.withColumn('Middle Initial', coalesce(col('Middle Initial'), lit('')))

# Concatenate the columns with first name first (case-insensitive) and remove trailing white spaces
df = df.withColumn('Full Name', trim(concat(lower(col('First Name')), lit(' '), lower(col('Middle Initial')), lit(' '), lower(col('Last Name')))))

# Replace null values with empty string for 'Visitee Last Name' and 'Visitee First Name' (case-insensitive)
df = df.withColumn('Visitee Last Name', coalesce(col('Visitee Last Name'), lit('')))
df = df.withColumn('Visitee First Name', coalesce(col('Visitee First Name'), lit('')))

df = df.withColumn("Start Date", split(col("Appointment Start Date"), " ").getItem(0))
df = df.withColumn("End Date", split(col("Appointment End Date"), " ").getItem(0))

# Concatenate the columns with first name first (case-insensitive) and remove trailing white spaces
df = df.withColumn('Visitor Full Name', trim(concat(lower(col('Visitee First Name')), lit(' '), lower(col('Visitee Last Name')))))

In [427]:
start_time = time.time()

In [428]:
# part i
most_frequent_visitors = df.groupBy('Full Name') \
    .count() \
    .orderBy(desc("count")) \
    .limit(10)

In [429]:
most_frequent_visitors.show()

+------------------+-----+
|         Full Name|count|
+------------------+-----+
|virginia s mcquade|  107|
|   elsie n stecker|  105|
|      amy k lanasa|  100|
| matilda n kuklish|   99|
|  michael n herzog|   98|
|       fern e sato|   94|
|   james w levings|   94|
|  russell a wilson|   93|
|  brian c turnmire|   87|
|    alan c prather|   86|
+------------------+-----+



In [430]:
# part ii
most_frequently_visited = df.groupBy('Visitor Full Name') \
    .count() \
    .orderBy(desc("count")) \
    .limit(10)

In [431]:
most_frequently_visited.show()

+-----------------+------+
|Visitor Full Name| count|
+-----------------+------+
|  visitors office|350842|
|            potus| 66081|
|        ed teleky| 12863|
|   amanda trocola|  5995|
|          dan via|  5072|
|     kevin ballen|  4841|
|  peyton schwartz|  4519|
|   gionelly mills|  4461|
|           vpotus|  3865|
|           room 1|  3267|
+-----------------+------+



In [432]:
most_frequent_combinations = df.groupBy("Full Name", 'Visitor Full Name') \
    .count() \
    .orderBy(desc("count")) \
    .limit(10)

In [433]:
most_frequent_combinations.show()

+--------------------+-----------------+-----+
|           Full Name|Visitor Full Name|count|
+--------------------+-----------------+-----+
|         fern e sato|          dan via|   69|
|  tiffany j ramseyer|     craig guyton|   63|
|     james w levings|       kolakowski|   63|
|    russell a wilson|          dan via|   60|
|    joshua a collins|        levi reed|   59|
|    brian c turnmire|          dan via|   57|
|      alan c prather|          dan via|   52|
|         eric d sabo|          dan via|   49|
|christopher e sch...|          dan via|   49|
|        aaron d clay|          dan via|   46|
+--------------------+-----------------+-----+



In [434]:
most_cancelled = cancelled.groupBy("Last Name", "First Name") \
    .count() \
    .orderBy(desc("count")) \
    .limit(10)

In [435]:
most_cancelled.show()

+----------+----------+-----+
| Last Name|First Name|count|
+----------+----------+-----+
|   MENDOZA|     JAMES|    4|
|    BROOKS|   VINCENT|    4|
|     EPPES|   DARNELL|    4|
|        NG|       YIU|    4|
|   MCQUAID|   MICHAEL|    4|
|    COYNER|      TONY|    4|
|     JAMES|    VERNON|    4|
| MALDONADO|       EVE|    3|
|   HINZMAN|     LARRY|    3|
|PIFFANELLI|  STEFANIA|    3|
+----------+----------+-----+



In [436]:
most_common_rooms  = df.groupBy('Meeting Room') \
    .count() \
    .orderBy(desc("count")) \
    .limit(10)

In [437]:
most_common_rooms.show()

+-------------------+------+
|       Meeting Room| count|
+-------------------+------+
|            EW TOUR|323354|
|           EW - RES| 43410|
|          WEST WING| 40549|
|         SOUTH LAWN| 37274|
|             EW 216| 12017|
|                350|  9850|
|                374|  7412|
|Bowling Alley - 037|  6749|
|         RES - Stat|  6199|
|             EW-RES|  5635|
+-------------------+------+



In [438]:
popular_party_sizes = df.groupBy('Total People') \
    .count() \
    .orderBy(desc('count')) \
    .limit(10)

In [439]:
popular_party_sizes.show()

+------------+-----+
|Total People|count|
+------------+-----+
|           4|76498|
|           6|60899|
|           2|60875|
|           3|50642|
|           5|50388|
|           1|39324|
|           7|16611|
|           8|12334|
|          10|10259|
|          50| 9883|
+------------+-----+



In [440]:
common_dates = df.groupby('Start Date', 'End Date') \
    .count() \
    .orderBy(desc('count')) \
    .limit(10)

In [441]:
common_dates.show()

+----------+----------+-----+
|Start Date|  End Date|count|
+----------+----------+-----+
|12/23/2022|12/23/2022|10110|
| 4/26/2023| 4/26/2023| 8754|
|12/13/2022|12/13/2022| 7638|
| 12/1/2022| 12/1/2022| 7117|
| 12/3/2022| 12/3/2022| 6995|
|  7/4/2022|  7/4/2022| 6967|
|12/18/2022|12/18/2022| 6807|
|12/17/2022|12/17/2022| 6428|
| 9/13/2022| 9/13/2022| 6295|
|10/31/2022|10/31/2022| 5560|
+----------+----------+-----+



In [442]:
# Times
end_time = time.time()
total = end_time - start_time
total

41.11058855056763

In [443]:
"""
TIMES
100%: 33.26685047149658
50%: 18.805326461791992
20%: 17.002009630203247
10%: 14.633686542510986
"""

'\nTIMES\n100%: 33.26685047149658\n50%: 18.805326461791992\n20%: 17.002009630203247\n10%: 14.633686542510986\n'